# Create Project with images by urls

A standart way to create new Project is to use Import page and upload pictures from hard drive. But this could cause troubles when you already have existing images on web server (i.e Amazon S3). Supervisely provides "Links" import plugin to add image to dataset without downloading any data - we just save link to remote image.

This cookbook implements the same functional. We will use popular movies posters from IMDB

**Input**:
- List of image urls

**Output**:
- New Project with images from input

<div style="display:flex">
     <div style="flex:1;padding-right:5px;">
          <img src="https://m.media-amazon.com/images/M/MV5BMjA0MDIwMDYwNl5BMl5BanBnXkFtZTcwMjY0Mzg4Mw@@._V1_SY1000_CR0,0,1350,1000_AL_.jpg">
     </div>
     <div style="flex:1;padding-left:5px;">
          <img src="https://m.media-amazon.com/images/M/MV5BMTc5Njg5Njg2MV5BMl5BanBnXkFtZTgwMjAwMzg5MTE@._V1_SY1000_CR0,0,1332,1000_AL_.jpg">
     </div>
</div>

## Configuration

Edit the following settings for your own case

In [1]:
import supervisely_lib as sly
import os
from slugify import slugify
from tqdm import tqdm

In [2]:
# Input urls

first_dataset_name = "dataset_01"
links_first_dataset = [
"https://m.media-amazon.com/images/M/MV5BMjA0MDIwMDYwNl5BMl5BanBnXkFtZTcwMjY0Mzg4Mw@@._V1_SY1000_CR0,0,1350,1000_AL_.jpg",
"https://m.media-amazon.com/images/M/MV5BMTc5Njg5Njg2MV5BMl5BanBnXkFtZTgwMjAwMzg5MTE@._V1_SY1000_CR0,0,1332,1000_AL_.jpg",
"https://m.media-amazon.com/images/M/MV5BMjA5ODU3NTI0Ml5BMl5BanBnXkFtZTcwODczMTk2Mw@@._V1_SX1777_CR0,0,1777,756_AL_.jpg"]

second_dataset_name = "dataset_02"
links_second_dataset = [
"https://m.media-amazon.com/images/M/MV5BMTY3MzMzMDgyMF5BMl5BanBnXkFtZTcwMzY0OTk1Mw@@._V1_SX1500_CR0,0,1500,999_AL_.jpg",
"https://m.media-amazon.com/images/M/MV5BMjk3NTYyMzc4Nl5BMl5BanBnXkFtZTcwODU3ODMzMw@@._V1_SY1000_CR0,0,1413,1000_AL_.jpg",
"https://m.media-amazon.com/images/M/MV5BNzM4OTkzMjcxOF5BMl5BanBnXkFtZTgwMTkxMjI1MTI@._V1_SX1777_CR0,0,1777,738_AL_.jpg",
"https://m.media-amazon.com/images/M/MV5BMTgwMTgxMTgwMl5BMl5BanBnXkFtZTcwODM5OTQyNw@@._V1_SY1000_SX1480_AL_.jpg",
"https://m.media-amazon.com/images/M/MV5BMjA3NTEwOTMxMV5BMl5BanBnXkFtZTgwMjMyODgxMzE@._V1_SX1500_CR0,0,1500,999_AL_.jpg",
"https://m.media-amazon.com/images/M/MV5BMzY0MDg5MTU5OV5BMl5BanBnXkFtZTcwMDUzNTIwNA@@._V1_SY1000_CR0,0,1488,1000_AL_.jpg",
"https://m.media-amazon.com/images/M/MV5BMTczMDE3NzU2M15BMl5BanBnXkFtZTcwNjkwNjM5NA@@._V1_.jpg"]

# Output Project
team_name = "jupyter_tutorials"
workspace_name = "cookbook"
project_name = "project_from_links"

# Obtain server address and your api_token from environment variables
# Edit those values if you run this notebook on your own PC
address = os.environ['SERVER_ADDRESS']
token = os.environ['API_TOKEN']

In [3]:
# Initialize API object
api = sly.Api(address, token)

## Verify input values

Test that context (team / workspace / project) exists

In [4]:
# Get IDs of team and workspace

team = api.team.get_info_by_name(team_name)
if team is None:
    raise RuntimeError("Team {!r} not found".format(team_name))

workspace = api.workspace.get_info_by_name(team.id, workspace_name)
if workspace is None:
    raise RuntimeError("Workspace {!r} not found".format(workspace_name))
    
print("Team: id={}, name={}".format(team.id, team.name))
print("Workspace: id={}, name={}".format(workspace.id, workspace.name))

Team: id=30, name=jupyter_tutorials
Workspace: id=76, name=cookbook


## Create empty Project

In [5]:
# check if project already exists. If yes - generate new free name
if api.project.exists(workspace.id, project_name):
    project_name = api.project.get_free_name(workspace.id, project_name)
    
# create remote project
project = api.project.create(workspace.id, project_name)
print("Project: id={}, name={}".format(project.id, project.name))

Project: id=1324, name=project_from_links


## Add remote images

We will use `api.image.upload_link` and `api.image.add_link` methods

In [6]:
for dataset_name, dataset_links in zip([first_dataset_name, second_dataset_name], [links_first_dataset, links_second_dataset]):
    dataset = api.dataset.create(project.id, dataset_name)
    with tqdm(total=len(dataset_links), desc="Process links") as progress_bar:
        for links in sly.batched(dataset_links):
            names = []
            for link in links:
                # Get image name from url and slugify it
                image_name = slugify(sly.fs.get_file_name(link))
                image_ext = sly.fs.get_file_ext(link)
                names.append("{}{}".format(image_name, image_ext))

            # Add new images to dataset via links
            # We will retrieve some fields like MIME type or file size
            # Image itself won't be stored on our servers
            api.image.upload_links(dataset.id, names, links)
            progress_bar.update(len(links))

Process links: 100%|██████████| 7/7 [00:00<00:00, 278.46it/s]


In [7]:
print("Project {!r} has been sucessfully uploaded".format(project.name))
print("Number of uploaded images: ", api.project.get_images_count(project.id))

Project 'project_from_links' has been sucessfully uploaded
Number of uploaded images:  10
